# GDVSpectra Threshold module tests

## Globals

In [10]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [11]:
# imports
import os
import random
import numpy as np
import xarray as xr
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [12]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [30]:
# setup general io
input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs'
output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\outputs'

# temp nc file and shape file for use when breaking ncs, shapefiles
temp_nc = os.path.join(input_folder, 'temp_nc.nc')     # temp nc file for use when breaking ncs
temp_shp = os.path.join(input_folder, 'temp_shp.shp')  # temp shp file for use when breaking shapefiles

# setup landsat cubes paths
ls_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\dwer_1_ls_90_20_like_odc_agg.nc",     # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\dwer_1_ls_90_20_like_odc_nonagg.nc",  # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\roy_1_ls_10_20_like_odc_agg.nc",      # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\roy_1_ls_10_20_like_odc_nonagg.nc",   # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\tute_1_ls_10_20_like_odc_agg.nc",     # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\tute_1_ls_10_20_like_odc_nonagg.nc",  # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\yandi_1_ls_90_20_like_odc_agg.nc",    # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\yandi_1_ls_90_20_like_odc_nonagg.nc"  # done all
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\dwer_1_s2_16_20_like_odc_agg.nc",     # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\dwer_1_s2_16_20_like_odc_nonagg.nc",  # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\roy_2_s2_16_21_like_odc_agg.nc",      # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\roy_2_s2_16_21_like_odc_nonagg.nc",   # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\tute_1_s2_18_20_like_odc_agg.nc",     # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\tute_1_s2_18_20_like_odc_nonagg.nc",  # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\yandi_2_s2_16_20_like_odc_agg.nc",    # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\yandi_2_s2_16_20_like_odc_nonagg.nc"  # done all
]

# set up presence/absence point shapefile
shps = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\pa_esri_albers.shp",                  # done all
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\pa_qgis_albers.shp",                  # done def
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_threshold\inputs\pa_esri_albers.gdb\pa_esri_albers"    # done def
]

### Set specific raw netcdf and shapefiles

In [61]:
# set top level netcdf and shapefile
nc_file = s2_cubes[-1]
shp_path = shps[0]

### Set up function to iterate corruptor and tests (without shapefile support)

In [62]:
def run_corruptors_through_tests(in_nc, nc_corrs, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed.
    """
    
    for nc_corr in nc_corrs:
        corr_name = nc_corr[0].__name__                   # name of current corruptor func
        corr_func, corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params

        # notify
        print('Corrupting NetCDF via: {}.\n'.format(corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            corr_func(**corr_params)
        except Exception as e:    
            print(e)
            print('Corruptor did not have enough data to work with. Skipping.\n')

        # iter each test func and apply to current corrupt nc
        for test in tests:
            test_nc_name = corr_name + '_' + test[0]    # name of current test nc
            test_func, test_params = test[1], test[2]   # pointer to test func and dict of params
            test_msg = test[3]
            
            # notify of test message
            print(test_msg)

            # create output nc file path and name and update params for in/out paths
            out_nc_file = os.path.join(output_folder, test_nc_name)
            
            # remove output nc if exists
            if os.path.exists(out_nc_file):
                os.remove(out_nc_file)
            
            # update params
            test_params.update({'in_nc_file': temp_nc, 'out_nc_file': out_nc_file})

            # perform current test
            try:
                # notify
                print('Performing test: {}.'.format(test_nc_name))

                # perform test, provide prints if requested
                if not verbose:
                    with io.capture_output() as cap:
                        test_func(**test_params)
                else:
                    test_func(**test_params)
                    print('\n')

            except Exception as e:    
                print(e)

        # notify
        print('All tests applied to corruptor NetCDF.\n\n')

### Set up function to iterate corruptor and tests (with shapefile support)

In [63]:
def run_corruptors_through_tests_with_shp(in_nc, nc_corrs, shp_corrs, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed. considers shapefile corruptors.
    """
    
    for nc_corr in nc_corrs:
        nc_corr_name = nc_corr[0].__name__                      # name of current corruptor func
        nc_corr_func, nc_corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params

        # notify
        print('Corrupting NetCDF via: {}.\n'.format(nc_corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            nc_corr_func(**nc_corr_params)
        except Exception as e:    
            print(e)
            print('NetCDF Corruptor did not have enough data to work with. Skipping.\n')
            
        # now, we iterate each shapefile corruptor for this corrupt nc!
        for shp_corr in shp_corrs:
            shp_corr_name = shp_corr[0].__name__                        # name of current corruptor func
            shp_corr_func, shp_corr_params = shp_corr[0], shp_corr[1]   # pointer to corruptor func and dict of params
            
            # notify
            print('Corrupting Shapefile via: {}.\n'.format(shp_corr_name) + '- ' * 30)
            
            # create temp shp and corrupt it with current corruptor
            try:
                if not verbose:
                    with io.capture_output() as cap:
                        shp_corr_func(**shp_corr_params)  # run current corruptor function
                else:
                    shp_corr_func(**shp_corr_params)
            
            except Exception as e:    
                print(e)
                print('Shapefile corruptor did not have enough data to work with. Skipping.\n')
            
            # iter each test func and apply to current corrupt nc with consideration to shape
            for test in tests:
                test_nc_name = nc_corr_name + '_' + test[0]    # name of current test nc
                test_func, test_params = test[1], test[2]      # pointer to test func and dict of params
                test_msg = test[3]

                # notify of test message
                print(test_msg)
                
                # create output nc file path and name and update params for in/out paths
                out_nc_file = os.path.join(output_folder, test_nc_name)

                # remove output nc if exists
                if os.path.exists(out_nc_file):
                    os.remove(out_nc_file)

                # update params
                test_params.update({'in_nc_file': temp_nc, 'out_nc_file': out_nc_file, 
                                    'in_occurrence_feat': temp_shp, 'in_pa_column': 'p_a'})

                # perform current test
                try:
                    # notify
                    print('Performing test: {}.'.format(test_nc_name))

                    # perform test, provide prints if requested
                    if not verbose:
                        with io.capture_output() as cap:
                            test_func(**test_params)
                    else:
                        test_func(**test_params)
                        print('\n')

                except Exception as e:    
                    print(e)

            # notify
            print('All tests applied to corruptor NetCDF.\n\n')



### Set up netcdf corruptor functions

In [84]:
# these are numerous netcdf corruptors. feed a raw nc in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through the tests below
def build_nc_corruptors(temp_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_nc}])

#     # func: remove x, y, time, spatial_ref coords
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'x'}])
#     #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'y'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'time'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

#     #func: remove like band vars
#     cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'like'}])

#     #func: limit number of years in various combos
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2018, 'e_year': 2020}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2020, 'e_year': 2020}])

#     #func: set all vars to nan
#     cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

#     #func: set all vars to zero
#     cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

#     #func: set all vars for 10 rand times to all nan
#     cs.append([test_funcs.set_nc_vars_random_all_nan, {'in_nc': temp_nc, 'num': 3}])

#     #func: strip all attrs from nc    
#     cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

#     #func: set vars in first and last time index to all nan
#     cs.append([test_funcs.set_end_times_to_all_nan, {'in_nc': temp_nc}])

#     #func: reduce whole nc to one random time slice
#     cs.append([test_funcs.reduce_to_one_scene, {'in_nc': temp_nc}])

#     #func: remove crs attribute
#     cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

#     #func: invalidate crs attribute
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': 'EPSG:4326'}])
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': ''}])

#     #func: remove nodatavals attribute
#     cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])

    return cs

nc_corruptors = build_nc_corruptors(temp_nc=temp_nc)

### Setup shapefile corruptor functions

In [85]:
# these are numerous shapefile corruptors. feed a raw shp in, break it, output as temp shp
# comment out any that are irrelevant
# each of these uncommented will be fed through some tests below
def build_shp_corruptors(shp_path, temp_shp):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    s = []

    # func: raw default dataset, no changes 
    s.append([test_funcs.shp_default, {'shp_path': shp_path, 'temp_shp': temp_shp}])

    # func: convert shapefile to wgs84
    s.append([test_funcs.project_shp_to_wgs84, {'shp_path': shp_path, 'temp_shp': temp_shp}])

    # func: subset rows to each study area seperately
    #s.append([test_funcs.subset_shp_to_area, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'a'}])
    #s.append([test_funcs.subset_shp_to_area, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'b'}])
    #s.append([test_funcs.subset_shp_to_area, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'c'}])
    #s.append([test_funcs.subset_shp_to_area, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'd'}])
    
    # func: strip shapefile of projection info
    s.append([test_funcs.strip_shp_proj_file, {'shp_path': shp_path, 'temp_shp': temp_shp}])

    # func: convert shapefile pres/abse field to text type
    s.append([test_funcs.convert_shp_pa_field_to_text, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a'}])
    
    # func: randomlly set specific num of pres/abse values to something else
    s.append([test_funcs.random_set_shp_p_a_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'num_rand_samples': 100, 'set_to_value': 3}])
    s.append([test_funcs.random_set_shp_p_a_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'num_rand_samples': 5, 'set_to_value': 7}])
    s.append([test_funcs.random_set_shp_p_a_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'num_rand_samples': 13, 'set_to_value': -1}])
    s.append([test_funcs.random_set_shp_p_a_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'num_rand_samples': 200, 'set_to_value': 5}])
    
    # func: randomlly set specific num of pres/abse values to null
    s.append([test_funcs.random_set_shp_p_a_null, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'num_rand_samples': 5}])
    
    # func: reduce shapefile pres/abse point number to specified number
    s.append([test_funcs.reduce_shp_pa_num_points, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'a', 'pa_column': 'p_a', 'num_points': 5}])    
    s.append([test_funcs.reduce_shp_pa_num_points, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'b', 'pa_column': 'p_a', 'num_points': 2}])    
    s.append([test_funcs.reduce_shp_pa_num_points, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'c', 'pa_column': 'p_a', 'num_points': 15}])    
    s.append([test_funcs.reduce_shp_pa_num_points, {'shp_path': shp_path, 'temp_shp': temp_shp, 'area_code': 'd', 'pa_column': 'p_a', 'num_points': 50}])    
        
    # remove all points from shapefile 
    s.append([test_funcs.remove_all_shp_points, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a'}])
    
    # set all points in shapefile pres/abse column to specific values
    s.append([test_funcs.set_all_shp_points_to_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'new_val': 1}])
    s.append([test_funcs.set_all_shp_points_to_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'new_val': 0}])
    s.append([test_funcs.set_all_shp_points_to_value, {'shp_path': shp_path, 'temp_shp': temp_shp, 'pa_column': 'p_a', 'new_val': 3}])
    
    return s

shp_corruptors = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)

## Run tests

### Test One: Use Median of All Images

In [86]:
def build_test_one_funcs(in_nc, temp_nc):
    """sets up test one functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': None,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': 2.0,                     # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []

    # func: default all times median
    msg = 'Running Test One: default use all dates (True).'
    params = inputs.copy()
    params.update({'in_aggregate': True})
    ts.append(['t_1_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
        
    # func: dont use all times median
    msg = 'Running Test One: dont use all dates (False) no date set.'
    params = inputs.copy()
    params.update({'in_aggregate': False})
    ts.append(['t_1_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: dont use all times median provide date
    msg = 'Running Test One: dont use all dates (False) date (2020) set.'
    params = inputs.copy()
    params.update({'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_1_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # try above with default shapefile now
    inputs.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a'})
    
    # func: all times median but with shapefile
    msg = 'Running Test One: use all dates (True) but with shapefile set.'
    params = inputs.copy()
    params.update({'in_aggregate': True})
    ts.append(['t_1_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: dont use all times median but with shapefile
    msg = 'Running Test One: dont use all dates (False) no date set but with shapefile set.'
    params = inputs.copy()
    params.update({'in_aggregate': False})
    ts.append(['t_1_d.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: dont use all times median provide date
    msg = 'Running Test One: dont use all dates (False) date (2020) set.'
    params = inputs.copy()
    params.update({'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_1_e.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
     
    return ts

### Test One: Run!

In [87]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_one_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corruptors, shp_corrs=shp_corrs,tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test One: default use all dates (True).
Performing test: nc_default_t_1_def.nc.
Running Test One: dont use all dates (False) no date set.
Performing test: nc_default_t_1_a.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test One: dont use all dates (False) date (2020) set.
Performing test: nc_default_t_1_b.nc.
Running Test One: use all dates (True) but with shapefile set.
Performing test: nc_default_t_1_c.nc.
Running Test One: dont use all dates (False) no date set but with shapefile set.
Performing test: nc_default_t_1_d.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test One: dont use all dates (False) date (2020) set.
Performing test: nc_default_t_1_e.nc.
All 

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test One: default use all dates (True).
Performing test: nc_default_t_1_def.nc.
Running Test One: dont use all dates (False) no date set.
Performing test: nc_default_t_1_a.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test One: dont use all dates (False) date (2020) set.
Performing test: nc_default_t_1_b.nc.
Running Test One: use all dates (True) but with shapefile set.
Performing test: nc_default_t_1_c.nc.
Presence/absence column does not contain just 1s and 0s.
Failed to execute (GDVSpectra_Threshold).

Running Test One: dont use all dates (False) no date set but with shapefile set.
Performing test: nc_default_t_1_d.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test One: dont use all dates (False) date (2020) set.
Performing test: nc_default_t_1_e.nc.
Presence/absence column does not contain just 1s and 0s.
Failed to execute (GDVSpec

### Test Two: Specific Years

In [70]:
def build_test_two_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': False,                 # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': 2,                       # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []

    # func: default specific year (first in netcdf) without shapefile
    msg = 'Running Test Two: default specific year ("") without shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': ''})
    ts.append(['t_2_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year random selected (2018) without shapefile
    msg = 'Running Test Two: random specific year (2018) without shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '2018'})
    ts.append(['t_2_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year that isnt in dataset without shapefile
    msg = 'Running Test Two: specific year not in dataset (1989) without shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '1989'})
    ts.append(['t_2_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year that isnt in dataset without shapefile
    msg = 'Running Test Two: specific years in dataset (2016, 2017) without shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '2016;2017'})
    ts.append(['t_2_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # try above with default shapefile now 
    inputs.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a'})
    
    # func: default specific year (first in netcdf) with shapefile
    msg = 'Running Test Two: default specific year ("") with shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': ''})
    ts.append(['t_2_d.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year random selected (2018) with shapefile
    msg = 'Running Test Two: random specific year (2019) with shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '2019'})
    ts.append(['t_2_e.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year that isnt in dataset with shapefile
    msg = 'Running Test Two: specific year not in dataset (1985) with shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '1985'})
    ts.append(['t_2_f.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: specific year that isnt in dataset with shapefile
    msg = 'Running Test Two: specific years not in dataset (1975, 1976, 1977) with shapefile.'
    params = inputs.copy()
    params.update({'in_specific_years': '1975;1976;1977'})
    ts.append(['t_2_g.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    

    return ts

### Test Two: Run!

In [71]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_two_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corrs, shp_corrs=shp_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Two: default specific year ("") without shapefile.
Performing test: nc_default_t_2_def.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test Two: random specific year (2018) without shapefile.
Performing test: nc_default_t_2_a.nc.
Running Test Two: specific year not in dataset (1989) without shapefile.
Performing test: nc_default_t_2_b.nc.
ERROR 000582: Error occurred during execution.

Running Test Two: specific years in dataset (2016, 2017) without shapefile.
Performing test: nc_default_t_2_c.nc.
Running Test Two: default specific year ("") with shapefile.
Performing test: nc_default_t_2_d.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test Two: random spec

Running Test Two: random specific year (2018) without shapefile.
Performing test: set_nc_vars_random_all_nan_t_2_a.nc.
Running Test Two: specific year not in dataset (1989) without shapefile.
Performing test: set_nc_vars_random_all_nan_t_2_b.nc.
ERROR 000582: Error occurred during execution.

Running Test Two: specific years in dataset (2016, 2017) without shapefile.
Performing test: set_nc_vars_random_all_nan_t_2_c.nc.
NetCDF is empty. Please download again.
Failed to execute (GDVSpectra_Threshold).

Running Test Two: default specific year ("") with shapefile.
Performing test: set_nc_vars_random_all_nan_t_2_d.nc.
Did not provide a specific year.
Failed to execute (GDVSpectra_Threshold).

Running Test Two: random specific year (2019) with shapefile.
Performing test: set_nc_vars_random_all_nan_t_2_e.nc.
NetCDF is empty. Please download again.
Failed to execute (GDVSpectra_Threshold).

Running Test Two: specific year not in dataset (1985) with shapefile.
Performing test: set_nc_vars_rand

### Test Three: Occurrence Points

In [72]:
def build_test_three_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': True,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Occurrence Points',        # type of trend analysis
        'in_std_dev': 2,                       # standard dev value
        'in_occurrence_feat': shp_path,        # occurrence shapefile
        'in_pa_column': 'p_a',                 # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []
    
    # func: default specific year (first in netcdf) without shapefile
    msg = 'Running Test Three: default aggregated with shapefile.'
    params = inputs.copy()
    params.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a'})
    ts.append(['t_3_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: default specific year (first in netcdf) without shapefile
    msg = 'Running Test Three: random specific year (2020) with shapefile.'
    params = inputs.copy()
    params.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a', 'in_specific_years': '2020'})
    ts.append(['t_3_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    return ts

### Test Three: Run!

In [73]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_three_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corrs, shp_corrs=shp_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Three: default aggregated with shapefile.
Performing test: nc_default_t_3_def.nc.
Running Test Three: random specific year (2020) with shapefile.
Performing test: nc_default_t_3_a.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: x
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Three: default aggregated with shapefile.
Performing test: remove_coord_t_3_def.nc.
Input NetCDF must have x, y coords.
Failed to execute (GDVSpectra_Threshold).

Running Test Three: random specific year (2020) with shapefile.
Performing test: remove_coord_t_3_a.nc.
Input NetCDF must have x, 

### Test Four: Standard Deviation Threshold

In [74]:
def build_test_four_funcs(in_nc, temp_nc):
    """sets up test four functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': True,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': None,                    # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []

    # func: default standard dev (2.0)
    msg = 'Running Test Four: default standard deviation (2.0).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0})
    ts.append(['t_4_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: standard dev is low 0.1
    msg = 'Running Test Four: standard deviation is low (0.1).'
    params = inputs.copy()
    params.update({'in_std_dev': 0.1})
    ts.append(['t_4_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

    # func: standard dev is high 10
    msg = 'Running Test Four: standard deviation is high (10).'
    params = inputs.copy()
    params.update({'in_std_dev': 10})
    ts.append(['t_4_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: standard dev is high 10
    msg = 'Running Test Four: standard deviation is out of range (15).'
    params = inputs.copy()
    params.update({'in_std_dev': 15})
    ts.append(['t_4_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: standard dev is lowest 0
    msg = 'Running Test Four: standard deviation is lowest (0).'
    params = inputs.copy()
    params.update({'in_std_dev': 0})
    ts.append(['t_4_d.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: standard dev is lowest None
    msg = 'Running Test Four: standard deviation is None.'
    params = inputs.copy()
    params.update({'in_std_dev': None})
    ts.append(['t_4_e.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    
    
    # func: standard dev is default but without aggregate
    msg = 'Running Test Four: standard deviation is default but no aggregation (using 2020).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0, 'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_4_f.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])        
    
    # try above with default shapefile now 
    inputs.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a'})
    
    # func: standard dev is 2.0 but with with shapefile
    msg = 'Running Test Four: default standard deviation with default shapefile.'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0})
    ts.append(['t_4_g.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg]) 
    
    return ts

### Test Four: Run!

In [75]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_four_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corrs, shp_corrs=shp_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Four: default standard deviation (2.0).
Performing test: nc_default_t_4_def.nc.
Running Test Four: standard deviation is low (0.1).
Performing test: nc_default_t_4_a.nc.
Running Test Four: standard deviation is high (10).
Performing test: nc_default_t_4_b.nc.
Threshold returned no values, try modifying threshold.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is out of range (15).
Performing test: nc_default_t_4_c.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 0 to 10.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is lowest (0).
Performing test: nc_default_t_4_d.nc.
Running Test Four: standard deviation is None.
Perform


Running Test Four: default standard deviation with default shapefile.
Performing test: remove_var_t_4_g.nc.
Input NetCDF has no data/variables/bands.
Failed to execute (GDVSpectra_Threshold).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: limit_years.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Limiting years to 2018 - 2020.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Four: default standard deviation (2.0).
Performing test: limit_years_t_4_def.nc.
Running Test Four: standard deviation is low (0.1).
Performing test: limit_years_t_4_a.nc.
Running Test Four: standard deviation is high (10).
Performing test: limit_years_t_4_b.nc.
Threshold returned no values, try modifying threshold.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is out of range (15).
Performing test: limit_years_t_4_c.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The v


Running Test Four: default standard deviation with default shapefile.
Performing test: set_nc_vars_random_all_nan_t_4_g.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: strip_nc_attributes.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Stripping attributes from nc dim
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Four: default standard deviation (2.0).
Performing test: strip_nc_attributes_t_4_def.nc.
NetCDF attributes not found. NetCDF must have attributes.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is low (0.1).
Performing test: strip_nc_attributes_t_4_a.nc.
NetCDF attributes not found. NetCDF must have attributes.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is high (10).
Performing test: strip_nc_attributes_t_4_b.nc.
NetCDF attributes not found. NetCDF must have attributes.
Failed to execute (GDVSpectra_Thresh


Running Test Four: standard deviation is None.
Performing test: invalidate_crs_attr_t_4_e.nc.
NetCDF CRS is not EPSG:3577. EPSG:3577 required.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: standard deviation is default but no aggregation (using 2020).
Performing test: invalidate_crs_attr_t_4_f.nc.
NetCDF CRS is not EPSG:3577. EPSG:3577 required.
Failed to execute (GDVSpectra_Threshold).

Running Test Four: default standard deviation with default shapefile.
Performing test: invalidate_crs_attr_t_4_g.nc.
NetCDF CRS is not EPSG:3577. EPSG:3577 required.
Failed to execute (GDVSpectra_Threshold).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: invalidate_crs_attr.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Invalidating crs attribute from nc dim with: 
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Four: default standard deviation (2.0).
Performing test: invalidate_crs_attr_t

### Test Five: If Nodata Handler (deprecated)

In [76]:
# def build_test_five_funcs(in_nc, temp_nc):
#     """sets up test five functions"""
    
#     # set default params for tool
#     inputs = {
#         'in_nc_file': '',                      # input nc (i.e. temp nc)
#         'out_nc_file': '',                     # output nc (i.e. t1a nc)
#         'in_aggregate': True,                  # use all dates
#         'in_specific_years': '',               # select specific years
#         'in_type': 'Standard Deviation',       # type of trend analysis
#         'in_std_dev': 2.0,                     # standard dev value
#         'in_occurrence_feat': '',              # occurrence shapefile
#         'in_pa_column': '',                    # presence/absence column
#         'in_remove_stray': True,               # remove salt and pepper
#         'in_convert_binary': True,             # binarise output into 1, 0
#         'in_add_result_to_map': True,          # add result to map
#     }
    
#     # set up list
#     ts = []

#     # func: default if no data is any
#     msg = 'Running Test Five: default if nodata is Any.'
#     params = inputs.copy()
#     params.update({'in_if_nodata': 'Any'})
#     ts.append(['t_5_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])

#     # func: if no data is All
#     msg = 'Running Test Five: if nodata is All.'
#     params = inputs.copy()
#     params.update({'in_if_nodata': 'All'})
#     ts.append(['t_5_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    

#     # func: if no data is ''
#     msg = 'Running Test Five: if nodata is "".'
#     params = inputs.copy()
#     params.update({'in_if_nodata': ''})
#     ts.append(['t_5_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    
    
#     # func: if no data is hello
#     msg = 'Running Test Five: if nodata is fake = Hello.'
#     params = inputs.copy()
#     params.update({'in_if_nodata': 'Hello'})
#     ts.append(['t_5_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])  
    
#     # try above with default shapefile now 
#     inputs.update({'in_type': 'Occurrence Points', 'in_occurrence_feat': shp_path, 'in_pa_column': 'p_a'})
    
#     # func: default if no data is any with shapefile
#     msg = 'Running Test Five: if no data is any with shapefile.'
#     params = inputs.copy()
#     params.update({'in_if_nodata': 'Any'})
#     ts.append(['t_5_d.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg]) 
    
#     # func: if no data is All with shapefile
#     msg = 'Running Test Five: if no data is all with shapefile.'
#     params = inputs.copy()
#     params.update({'in_if_nodata': 'All'})
#     ts.append(['t_5_e.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg]) 
    
#     return ts

### Test Five: Run! (deprecated)

In [77]:
# no longer used!

# build lsit of nc corruptors and tests to iterate
#nc_corrs = build_nc_corruptors(temp_nc)
#tests = build_test_five_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
#run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

### Test Six: Remove Stray

In [78]:
def build_test_six_funcs(in_nc, temp_nc):
    """sets up test six functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': True,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': 2.0,                     # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []

    # func: default remove stray
    msg = 'Running Test Six: default remove stray (True).'
    params = inputs.copy()
    params.update({'in_remove_stray': True})
    ts.append(['t_6_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: default remove stray but without aggregate
    msg = 'Running Test Six: remove stray is default but no aggregation (using 2020).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0, 'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_6_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    
    
    # func: remove stray is false
    msg = 'Running Test Six: remove stray is False.'
    params = inputs.copy()
    params.update({'in_remove_stray': False})
    ts.append(['t_6_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: false remove stray but without aggregate
    msg = 'Running Test Six: remove stray is False but no aggregation (using 2020).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0, 'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_6_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg]) 
    
    return ts

### Test Six: Run!

In [79]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_six_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corrs, shp_corrs=shp_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Six: default remove stray (True).
Performing test: nc_default_t_6_def.nc.
Running Test Six: remove stray is default but no aggregation (using 2020).
Performing test: nc_default_t_6_a.nc.
Running Test Six: remove stray is False.
Performing test: nc_default_t_6_b.nc.
Running Test Six: remove stray is False but no aggregation (using 2020).
Performing test: nc_default_t_6_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: x
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Six: default remove stray (True).
Performing test: remove_coord_t_6_def.nc.
Input NetCDF must have x


Corrupting NetCDF via: set_nc_vars_random_all_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting 3 random times in nc var data to all nan
['2018-01-01T00:00:00.000000000' '2018-01-01T00:00:00.000000000'
 '2017-01-01T00:00:00.000000000']
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Six: default remove stray (True).
Performing test: set_nc_vars_random_all_nan_t_6_def.nc.
Running Test Six: remove stray is default but no aggregation (using 2020).
Performing test: set_nc_vars_random_all_nan_t_6_a.nc.
Running Test Six: remove stray is False.
Performing test: set_nc_vars_random_all_nan_t_6_b.nc.
Running Test Six: remove stray is False but no aggregation (using 2020).
Performing test: set_nc_vars_random_all_nan_t_6_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: strip_nc_attributes.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Stripping attributes from nc dim
Corrup

### Test Seven: Convert to Binary

In [80]:
def build_test_seven_funcs(in_nc, temp_nc):
    """sets up test seven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': True,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': 2.0,                     # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []

    # func: default convert binary
    msg = 'Running Test Seven: default convert binary (True).'
    params = inputs.copy()
    params.update({'in_convert_binary': True})
    ts.append(['t_7_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: default convert binary but without aggregate
    msg = 'Running Test Seven: convert binary is default but no aggregation (using 2020).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0, 'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_7_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    
    
    # func: convert binary is false
    msg = 'Running Test Seven: convert binary is False.'
    params = inputs.copy()
    params.update({'in_convert_binary': False})
    ts.append(['t_7_b.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: false convert binary but without aggregate
    msg = 'Running Test Seven: convert binary is False but no aggregation (using 2020).'
    params = inputs.copy()
    params.update({'in_std_dev': 2.0, 'in_aggregate': False, 'in_specific_years': '2020'})
    ts.append(['t_7_c.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg]) 
    
    return ts

### Test Seven: Run!

In [81]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
shp_corrs = build_shp_corruptors(shp_path=shp_path, temp_shp=temp_shp)
tests = build_test_seven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_shp(in_nc=nc_file, nc_corrs=nc_corrs, shp_corrs=shp_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Seven: default convert binary (True).
Performing test: nc_default_t_7_def.nc.
Running Test Seven: convert binary is default but no aggregation (using 2020).
Performing test: nc_default_t_7_a.nc.
Running Test Seven: convert binary is False.
Performing test: nc_default_t_7_b.nc.
Running Test Seven: convert binary is False but no aggregation (using 2020).
Performing test: nc_default_t_7_c.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: x
Corrupting Shapefile via: shp_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Running Test Seven: default convert binary (True).
Performing test: remove_coord_t_7_def.nc.
Inpu

Running Test Seven: convert binary is False but no aggregation (using 2020).
Performing test: remove_nodatavals_attr_t_7_c.nc.
NetCDF nodatavals attribute not found.
Failed to execute (GDVSpectra_Threshold).

All tests applied to corruptor NetCDF.




### Test Eight: Add To Map

In [82]:
def build_test_eight_funcs(in_nc, temp_nc):
    """sets up test eight functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                      # input nc (i.e. temp nc)
        'out_nc_file': '',                     # output nc (i.e. t1a nc)
        'in_aggregate': True,                  # use all dates
        'in_specific_years': '',               # select specific years
        'in_type': 'Standard Deviation',       # type of trend analysis
        'in_std_dev': 2.0,                     # standard dev value
        'in_occurrence_feat': '',              # occurrence shapefile
        'in_pa_column': '',                    # presence/absence column
        'in_remove_stray': True,               # remove salt and pepper
        'in_convert_binary': True,             # binarise output into 1, 0
        'in_add_result_to_map': True,          # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default add result to map (True)
    msg = 'Running Test Eight: default add result to map (True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_8_def.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])
    
    # func: add result to map is False
    msg = 'Running Test Eight: add result to map is False.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_8_a.nc', arcpy.GDVSpectra_Threshold_toolbox , params, msg])    
     
    return ts

### Test Eight: Run!

In [83]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eight_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Eight: default add result to map (True).
Performing test: nc_default_t_8_def.nc.
Running Test Eight: add result to map is False.
Performing test: nc_default_t_8_a.nc.
All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: x
Running Test Eight: default add result to map (True).
Performing test: remove_coord_t_8_def.nc.
Input NetCDF must have x, y coords.
Failed to execute (GDVSpectra_Threshold).

Running Test Eight: add result to map is False.
Performing test: remove_coord_t_8_a.nc.
Input NetCDF must have x, y coords.
Failed to execute (GDVSpectra_Threshold).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: time
Running Test Eight: d


Running Test Eight: add result to map is False.
Performing test: remove_nodatavals_attr_t_8_a.nc.
NetCDF nodatavals attribute not found.
Failed to execute (GDVSpectra_Threshold).

All tests applied to corruptor NetCDF.


